# Single panel model

Introduction:
------------------

Single panel model:  In this model, power output is calculated from a single PV panel per grid cell per time-step. This panel is oriented horizontally (i.e., flat) without considering any shading or slope. The output from this single PV panel is then scaled to the entire neighbourhood and reduced due to effects of geometric placement and shading.

Background:
-----------------
This model was developed using LiDAR and DSM data. Forty simulation areas have been generated in which 25 training areas are used to analyze the relationships between PV power production potential and urban forms and later the relationships were evaluated using 15 evaluation areas.

Parametrized parameters, namely placement efficiency and shading efficiency are generated from the training areas. Placement efficiency was used to accurately calculate geometrically placed PV panels for an urban area without actually placing PV panels on rooftop building segments. Shading efficiency calculates the PV power output with shading losses instead of using the SEBE model. 

These parametrized equations are used in Big PV model. With the placement and shading efficiency, PV power output for an urban area can be calculated in a simple and fast manner.

Significance:
-----------------
1. With fewer input parameters, can rapidly estimate the PV potential over an entire year.
2. Parametrized equations used in this model assist in calculating geometrically placed PV panels and account for shading losses.

Assumptions/Limitations:
------------------------------------
1. Assumes all rooftop building segments are equipped with PV panels neglecting safety, socio-economic and environmental factors.
2. Placement efficiency loss was calculated based on a standard PV panel having dimensions (1m x 1.6m)

Output:
-----------

Hourly PV power output [MW km-2] for an urban area

In [124]:
    """

    Input Parameters
    ----------

        A_G          : Total plan area of grid cell [m2]
        A_R          : Projected plan area of all rooftop segments in a grid cell [m2]
        λ_p          : Projected plan area fraction of building segments (A_R/A_G)
        l_s          : Average building segment length [m] is the typical distance of the length of the roof, 
                       which tells you how far the shadows get on the roof and is calculated 
                       by taking the square root of the average segment area.
        f_f          : Fraction of projected rooftop segment areas that are flat (0-1)
        f_p          : Scaling fator coefficient(f_p = 1 if all the rooftop segments are covered with panels)
        
        η_p          : Efficiency of PV panel [%]
        α_T          : Temperature coefficient of PV panel [1/K]
        η_W          : Wiring losses

        metadata     : Input meteorological data specifically formatted using 
                       UMEP -> Pre-processing -> Meteorological data -> Prepare existing data
                       Here, a sample Random Meteorological Day(RMD) for the month of June has been given as an example.
                       Annual weather data for the city of Berlin has been attached for reference.
        sol_angles   : Sun position angles for the urban area.
                       Here, solar angles for the RMD - June has been given as example    
        
        
        Note: 1. In this script, l, w, η_p, α_T values has been taken for a standard PV module. 
              2. λ_p, l_s, f_f and A_G values has been taken for an training area in the city of Berlin
              3. Input files and parameters can be found under the folder "Datasets" in Github
    """

import numpy as np
import pandas as pd
import math
import pvlib  

# Urban form parameters
A_G         = 265200
λ_p         = 0.398  
l_s         = 9.36  
f_f         = 0.413  
f_p         = 1


# PV panel characteristics
η_p         = 0.177 
α_T         = -0.0039
η_W         = 0.8

# Meteorological data
metdata     = np.loadtxt("june_RMD.txt", skiprows=1, dtype=float).reshape((-1, 24)) # Input file is located under Datasets folder
sol_angles  = pd.read_csv(f'june_angles_RMD.csv',index_col=0) # Input file is located under Datasets folder

In [126]:
  """
    Constants
    ---------
        T_r          : Reference PV cell temperature, default value = 25 °C
        k_t          : Reduction factor, default value = 0.05 °C/(W/m2)
            
            Reduction factor (kT) a constant defined as “expressing the changes in module performance due to 
            differences in the module’s actual , and nominal operating temperatures” (Ramirez Camargo et al., 2015). 
            A constant value for kT = 0.05 °C/(W/m2) was suggested by various authors (King et al., 2004), 
            (Ramirez Camargo et al., 2015) and also used in the PV-GIS web service for 
            building integrated systems.
            
            References:
            ----------
            
            Ramirez Camargo, L., Zink, R., Dorner, W., & Stoeglehner, G. (2015). 
            Spatio-temporal modelling of roof-top photovoltaic panels for improved technical potential assessment 
            and electricity peak load offsetting at the municipal scale. Computers, Environment, and Urban Systems, 
            52, 58–69. https://doi.org/10.1016/j.compenvurbsys.2015.03.002
            
            King, D. L., Boyson, W. E., & Kratochvill, J. A. (2004). Photovoltaic Array Performance Model. December.
            
            
  """

T_r      = 25
k_t      = 0.05

In [127]:
  """
    Theoretical PV panel area
    ---------------------
    
        A_T_1P  : PV panel area is calculated using rooftop building segment area without considering the geometry, tilt, 
                  and aspect of the building segment. [m2]
        
  """
A_R             = λ_p * A_G
A_T_1P          = A_R

In [128]:
  """
    Parametrized Equations
    ----------------------
    
        ϵ_p          : Placement efficiency is the ratio of geometrically installed PV panels
                       to the theoretical PV panels.
                
        ϵ_s_dir      : Direct shading efficiency loss factor is the reduction of direct POA irradiance due to shading 
                       and is estimated by comparing the direct POA irradiance for Model 1P with building resolving model
                       
        ϵ_s_dif      : Diffuse shading efficiency loss factor is the reduction of diffuse POA irradiance due to shading 
                       and is estimated by comparing the diffuse POA irradiance for Model 1P with building resolving model
                
  """
    
ϵ_p     = (0.4435 * f_f + 0.0971)
ϵ_s_dir = (-0.00043 * l_s - 0.00461)
ϵ_s_dif = (-0.00562 * l_s + 0.43536)

In [129]:
  """
    Geometric PV panel area
    -----------------------
    
        A_3_1P : Area of PV panels that could be in a grid cell considering parametrized placement efficiency loss [m2]

  """
A_3_1P = A_T_1P * (1 - ϵ_p)

In [130]:
pv_power = pd.DataFrame()


def single_panel(panel_area):
    
    """
    Function
    --------
    
        For calculating PV power output using direct and diffuse irradiance and parametrized equations

    """

    for i in range(metdata.shape[0]):

        if metdata[[i], 14] > 0: # Considered only the hours when Global horizontal irradiance > 0 W/m2

            γ        = sol_angles.loc[int(metdata[[i], 2]), 'Solar_Elevation']
            poa_dir  = metdata[[i], 22] * (1 - (ϵ_s_dir * γ + 1))
            poa_dif  = metdata[[i], 21] * (1 - ϵ_s_dif)
            metdata1 = metdata[[i], :]

            if γ > 0: # Considered only Sunshine hours (Solar elevation > 0)
                pv_power.loc[i,f'Year']      = int(metdata1[0,0])
                pv_power.loc[i,f'Day']       = int(metdata1[0,1])
                pv_power.loc[i,f'Hour']      = int(metdata1[0,2])
                pv_power.loc[i,f'direct [MW km-2]']    = poa_dir * η_p * (1 + α_T * ((metdata[[i], 11] + k_t * poa_dir) - T_r)) * panel_area * η_W * f_p / A_G 
                pv_power.loc[i,f'diffuse [MW km-2]']   = poa_dif * η_p * (1 + α_T * ((metdata[[i], 11] + k_t * poa_dif) - T_r)) * panel_area * η_W * f_p / A_G 

    return pv_power

modelled_PVO = single_panel(A_3_1P)

pv_power['Total_PVP [MW km-2]'] = (pv_power['direct [MW km-2]'] + pv_power['diffuse [MW km-2]'])

In [131]:
"""
    Output
    -------
        Hourly PV power output [MW km-2] for an urban area
        
"""
pv_power

,Year,Day,Hour,direct [MW km-2],diffuse [MW km-2],Total_PVP [MW km-2]
5,2019.0,167.0,5.0,0.007856,0.674301,0.682157
6,2019.0,171.0,6.0,0.270086,1.445122,1.715208
7,2019.0,160.0,7.0,1.163209,2.434763,3.597972
8,2019.0,152.0,8.0,1.729760,3.353128,5.082888
9,2019.0,175.0,9.0,5.587079,2.875020,8.462099
10,2019.0,179.0,10.0,0.909695,6.411023,7.320719
11,2019.0,169.0,11.0,10.652533,3.500598,14.153131
12,2019.0,174.0,12.0,12.746680,3.528719,16.275400
13,2019.0,156.0,13.0,11.015279,4.384120,15.399399
14,2019.0,165.0,14.0,12.198025,3.377707,15.575732
